In [ ]:
import logging
import asyncio
import pandas as pd
import polars as pl

from datetime import datetime, timedelta

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.config.enumerations import Channels
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections import Credentials, InfluxCredentials

from tastytrade.messaging.processors.influxdb import TelegrafHTTPEventProcessor

from tastytrade.connections.subscription import RedisSubscriptionStore

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2025, 1, 1)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

In [ ]:
credentials = Credentials(env="Live")
dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
await dxlink.open(credentials=credentials)

In [3]:
influx_user = InfluxCredentials()
influxdb = influxdb_client.InfluxDBClient(**InfluxCredentials().__dict__)

In [ ]:
for handler in dxlink.router.handler.values():
    handler.add_processor(TelegrafHTTPEventProcessor())

# checks
dxlink.router.handler[Channels.Candle].processors

# Candle Subscriptions

In [ ]:
start_time = datetime(2025, 2, 28)

# for symbol in ["NVDA"]:
# for symbol in ["BTC/USD:CXTALP", "NVDA", "QQQ", "SPY", "SPX"]:

for symbol in ["BTC/USD:CXTALP", "NVDA", "QQQ", "SPY", "SPX"]:
    for interval in ["1d", "1h", "30m", "15m", "5m", "m"]:
        coroutine = dxlink.subscribe_to_candles(
            symbol=symbol,
            interval=interval,
            from_time=start_time,
        )
        await asyncio.wait_for(coroutine, timeout=10)

# Market Data feeds

In [ ]:
# symbols = ["NVDA"]
# symbols = ["SPX"]
# symbols = ["SPX", "NVDA", "SPY", "QQQ"]
symbols = ["BTC/USD:CXTALP", "SPX", "NVDA", "SPY", "QQQ"]
await dxlink.subscribe(symbols)

# Forward fill overnight gap

In [7]:
from tastytrade.utils.time_series import forward_fill

for symbol in ["BTC/USD:CXTALP", "NVDA", "QQQ", "SPY", "SPX"]:
    # for symbol in ["SPX"]:
    for interval in ["1d", "1h", "30m", "15m", "5m", "1m"]:
        event_symbol = f"{symbol}{{={interval}}}"
        logging.debug("Forward-filling %s", event_symbol)
        forward_fill(symbol=event_symbol, lookback_days=5)

In [ ]:
symbol = "SPX{=m}"
symbol = "BTC/USD:CXTALP{=5m}"

dxlink.router.handler[Channels.Candle].processors["feed"].frames[f"{symbol}"].tail(5).sort(
    by="time", descending=True
)

In [ ]:
dxlink.router.handler[Channels.Candle].processors["feed"].frames[f"{symbol}"].shape

In [10]:
# TODOS

# [x] Widen the plot
# [x] Remove the scroller at the bottom
# [x] move the legent and remove Price (that is obvious)
# [x] Add MACD
# [x] Fix HULL - Align w/ candlesticks
# [x] ERROR if no study data found

# [ ] Add RSI
# [ ] Add Volume Profile (?? ... /ES, SPY, etc)
# [ ] Add velocity metric

In [11]:
from tastytrade.analytics.visualizations.plots import (
    plot_macd_with_hull,
    HorizontalLine,
    VerticalLine,
)
from tastytrade.analytics.indicators.momentum import macd
from tastytrade.messaging.models.events import CandleEvent
import re

from tastytrade.providers.market import MarketDataProvider
from tastytrade.analytics.visualizations.utils import get_opening_range

import pytz

# Create timezone-aware datetime objects

day = 28

et_tz = pytz.timezone("America/New_York")
market_open = datetime(2025, 2, day, 9, 30, tzinfo=et_tz)
morning_end = datetime(2025, 2, day, 11, 30, tzinfo=et_tz)
lunch_end = datetime(2025, 2, day, 13, 30, tzinfo=et_tz)
market_close = datetime(2025, 2, day, 16, 0, tzinfo=et_tz)

streamer = MarketDataProvider(dxlink, influxdb)

start = datetime(2025, 2, day, 9) + timedelta(hours=5)
stop = datetime(2025, 2, day, 17) + timedelta(hours=5)

In [12]:
# import importlib
# import tastytrade.analytics.visualizations.plots
# importlib.reload(tastytrade.analytics.visualizations.plots)
# plot_macd_with_hull = tastytrade.analytics.visualizations.plots.plot_macd_with_hull

In [13]:
candle_symbol = "SPX{=m}"

prior_day: CandleEvent = CandleEvent(
    **(
        streamer.download(
            symbol=re.sub(r"\{=.*?\}", "{=d}", candle_symbol),
            start=start.date() + timedelta(days=-1),
            stop=start.date(),
            debug_mode=True,
        )
        .to_dicts()
        .pop()
    )
)

In [ ]:
or5 = await get_opening_range(
    streamer,
    "SPX{=m}",
    5,
    date=start.date(),
)

or15 = await get_opening_range(
    streamer,
    "SPX{=m}",
    15,
    date=start.date(),
)

or30 = await get_opening_range(
    streamer,
    "SPX{=m}",
    30,
    date=start.date(),
)

In [15]:
levels = [
    HorizontalLine(
        price=prior_day.close,
        # label="Prior Close",
        label=int(prior_day.close),
        color="#FF66FE",  # Orange
        line_dash="dot",
        label_font_size=10.5,
        # extend_to_end=False,
        # start_time=market_open,
        # end_time=lunch_end - timedelta(minutes=15),
        opacity=0.45,
    ),
    HorizontalLine(
        price=prior_day.high,
        # label="Prior High",
        label=int(prior_day.high),
        color="#4CAF50",  # Green
        line_dash="dot",
        label_font_size=10.5,
        opacity=0.45,
    ),
    HorizontalLine(
        price=prior_day.low,
        # label="Prior High",
        label=int(prior_day.low),
        color="#F44336",  # Red
        line_dash="dot",
        label_font_size=10.5,
        opacity=0.45,
    ),
    HorizontalLine(
        price=or5.high,
        start_time=market_open,
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.75,
    ),
    HorizontalLine(
        price=or5.low,
        start_time=market_open,
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.75,
    ),
    HorizontalLine(
        price=or15.high,
        start_time=market_open + timedelta(minutes=15),
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.45 if or15.high != or5.high else 0.0,
    ),
    HorizontalLine(
        price=or15.low,
        start_time=market_open + timedelta(minutes=15),
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.45 if or15.low != or5.low else 0.0,
    ),
    HorizontalLine(
        price=or30.high,
        start_time=market_open + timedelta(minutes=30),
        color="#4CAF50",  # Green
        line_dash="dot",
        opacity=0.45 if or30.high != or15.high else 0.0,
    ),
    HorizontalLine(
        price=or30.low,
        start_time=market_open + timedelta(minutes=30),
        color="#4CAF50",  # Green
        line_dash="dot",
        opacity=0.45 if or30.low != or15.low else 0.0,
    ),
]

In [16]:
executions = [
    VerticalLine(
        time=datetime(2025, 2, 26, 14, 30) + timedelta(hours=1, minutes=50),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 2, 26, 14, 30) + timedelta(hours=4, minutes=50),
        color="#555555",
        line_dash="dot",
        label="Close",
    ),
    VerticalLine(
        time=datetime(2025, 2, 27, 14, 30) + timedelta(minutes=20),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    VerticalLine(
        time=datetime(2025, 2, 28, 14, 30) + timedelta(hours=1, minutes=40),
        color="#555555",
        line_dash="dot",
        label="Open\nBPS",
    ),
]

In [ ]:
candles: pl.DataFrame = streamer.download(
    symbol=candle_symbol,
    start=start,
    stop=stop,
    debug_mode=True,
)

df_macd = macd(candles, prior_close=prior_day.close, fast_length=12, slow_length=26, macd_length=9)

plot_macd_with_hull(
    df_macd,
    pad_value=prior_day.close,
    start_time=start,
    end_time=stop,
    horizontal_lines=levels,
    vertical_lines=executions,
)

In [ ]:
candles_5m: pl.DataFrame = streamer.download(
    symbol=candle_symbol.replace("m", "5m"),
    start=start,
    stop=stop,
    debug_mode=True,
)

df_macd_5m = macd(
    candles_5m, prior_close=prior_day.close, fast_length=12, slow_length=26, macd_length=9
)
plot_macd_with_hull(
    df_macd_5m,
    pad_value=prior_day.close,
    start_time=start,
    end_time=stop,
    horizontal_lines=levels,
    vertical_lines=executions,
)

In [ ]:
await dxlink.close()